# Projekt ZUM - streszczanie tekstu

In [1]:
!pip install datasets

from datasets import load_dataset
ds = load_dataset("abisee/cnn_dailymail", "3.0.0")


[notice] A new release of pip is available: 24.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


### Czyszczenie tekstu

In [2]:
import re

def clean_text(text):
  text = re.sub(r'http[s]?://\S+', '', text)
  text = text.strip().replace('\n', ' ').replace('\r', ' ')
  text = re.sub(r'\s+', ' ', text)
  return text

train_data = ds['train']
train_data = train_data_map = train_data.map(lambda x: {'article': clean_text(x['article']),
                                                        'highlights': clean_text(x['highlights'])})
val_data = ds['validation'].map(lambda x: {'article': clean_text(x['article']),
                                           'highlights': clean_text(x['highlights'])})
test_data = ds['test'].map(lambda x: {'article': clean_text(x['article']),
                                      'highlights': clean_text(x['highlights'])})

train_data[:2], val_data[:2], test_data[:2]

({'article': ['LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office ch

In [3]:
!pip install transformers torch


[notice] A new release of pip is available: 24.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
from transformers import AutoTokenizer

model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_data(examples):
    inputs = [f"summarize: {article}" for article in examples['article']]
    targets = examples['highlights']

    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=150,
            truncation=True,
            padding="max_length"
        )

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

train_data_tokenized = train_data.map(
    tokenize_data,
    batched=True,
    remove_columns=['article', 'highlights'],
    batch_size=1000
)

val_data_tokenized = val_data.map(
    tokenize_data,
    batched=True,
    remove_columns=['article', 'highlights'],
    batch_size=1000
)

test_data_tokenized = test_data.map(
    tokenize_data,
    batched=True,
    remove_columns=['article', 'highlights'],
    batch_size=1000
)

train_data_tokenized = train_data_tokenized.with_format("torch")
val_data_tokenized = val_data_tokenized.with_format("torch")
test_data_tokenized = test_data_tokenized.with_format("torch")

print(train_data_tokenized[0])


{'id': '42c027e4ff9730fbb3de84c1af0d2c506e41c3e4', 'input_ids': tensor([21603,    10,   301, 24796,  4170,     6,  2789,    41, 18844,    61,
         1636,  8929, 16023,  2213,  4173,  6324, 12591,    15, 11391,   592,
           12,     3,     9,  2196,  3996,  1755,   770,  8785,   591, 11039,
          770,    61, 13462,    38,     3,    88,  5050,   507,    30,  2089,
            6,    68,     3,    88, 10419,     7,     8,   540,   751,    31,
           17,  4061,     3,     9, 10783,    30,   376,     5,  4173,  6324,
        12591,    15,    38,  8929, 16023,    16,    96, 15537,   651, 16023,
           11,     8,  5197,    13,     8, 12308,   121,   304,     8, 19142,
           13, 29517,  6710,   343,     7,   300,     8,   296,     6,     8,
         1021,  7556,   845,     3,    88,    65,   150,  1390,    12,  9030,
           17,   449,   112,  1723,   550,    30,  1006,  2948,     6,  3281,
           11, 17086,  2251,     5,    96,   196,   278,    31,    17,   515,
